### The is a notebook to get popularTimes factors during week days for  a single place

In [6]:
import pandas as pd
import time
import json
import datetime
import pprint
from selenium import webdriver
from datetime import datetime
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains

# global str
# string= "Currently 27% busy, usually 33% busy."
#https://h6f1x.sse.codesandbox.io/
import re
getPlaceIDURL='https://h6f1x.sse.codesandbox.io/' #this url is unstalbe and sometimes get changed
###############################################
#loading URL_list
df = pd.read_csv ('TimhortonsURL10.csv')
# print (df)
print(df.info())
df=df.drop_duplicates(subset = ["URL"])
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   URL     5 non-null      object
dtypes: object(1)
memory usage: 168.0+ bytes
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 4
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   URL     5 non-null      object
dtypes: object(1)
memory usage: 80.0+ bytes
None


In [ ]:
import pandas as pd
import time
import json
import datetime
import pprint
from selenium import webdriver
from datetime import datetime
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains

# global str
# string= "Currently 27% busy, usually 33% busy."
import re
# getPlaceIDURL='https://googlemaps.github.io/js-samples/dist/samples/places-placeid-finder/index.html'
getPlaceIDURL='https://h6f1x.sse.codesandbox.io/' #this url is unstalbe and sometimes get changed
###############################################
#loading URL_list
df = pd.read_csv ('TimhortonsURL10.csv')
# print (df)
url_list= df["URL"].values
# print(url_list)
#####################################################################
#opernWebdriver
browser = webdriver.Firefox()
######################################################################
weekDay=["Sunday","Monday","Tuesday","Wednesday","Thursday","Friday","Saturday"]
# a Python object (dict):

########################################### prepare document to collect the data
document = {}
document["GoogleID"]="213424325432sdfdg"
document["currently-occupied-by"]= 112
document["name"]="Tim Hortons"
document["address"]="195 Westney Rd S, Ajax, ON L1S 2C9"

for i in weekDay:
    document[i]={}
#################################################
browser.get('https://www.google.ca/maps/place/Tim+Hortons/data=!4m5!3m4!1s0x89d51e6c109c7bb9:0xe862cf9d09617ef7!8m2!3d43.8861392!4d-78.945305?authuser=0&hl=en&rclk=1')
browser.execute_script("window.scrollTo(0, 20)") 
#############################################################
#get the addresss
data_item_id=browser.find_elements(By.XPATH,"//button[@data-item-id='address']")

for div in  data_item_id:
    if div.get_attribute('aria-label'):
#             print(div.get_attribute('aria-label'))
            string=div.get_attribute('aria-label')
#             print(string[string.find(": ")+2:])
            string= string[string.find(": ")+2:]+', Canada'
            document["address"]=string
            print(string)

#############################################################
div_tag =browser.find_elements(By.TAG_NAME , "div")
weekIndex=-1
lastTime="p.m"
for div in  div_tag:
    if div.get_attribute('aria-label'):
        divString=div.get_attribute('aria-label')
        if divString:
            ####################################
            if "a.m" in divString and "p.m" in lastTime:
                weekIndex=weekIndex+1 ##a new day    
            #################################### case of currently time         
            if "Currently" in divString:
                busy= re.findall(r'\d+',divString)[1]
                now = datetime.now()
                AMorPM=" a.m"
                current_time = now.strftime("%H")
                if int(current_time)>12:
                    current_time=int(current_time)%12
                    current_time=str(current_time)
                    AMorPM=" p.m"
                theTime=current_time+AMorPM
                document[weekDay[weekIndex]][theTime]=busy
                lastTime=theTime
            if "a.m" in divString or "p.m" in divString:
                busy=int(divString[0:divString.find("%")])
                if "a.m" in divString:
                    AMorPM = " a.m"
                else:
                    AMorPM = " p.m"
                timepm= re.findall(r'\d+',divString)[1]
                theTime= timepm+AMorPM
                document[weekDay[weekIndex]][theTime]=busy
                lastTime=theTime
pprint.pprint(document)

#############################################
#get PlaceID using the saved Address
browser.get(getPlaceIDURL)
input_text= browser.find_element(By.XPATH,"//input[@id='pac-input']")
print(input_text.get_attribute('placeholder'))
time.sleep(10)
input_text.click()
input_text.send_keys(document["address"])
time.sleep(3)
input_text.send_keys(Keys.ARROW_DOWN)
input_text.send_keys(Keys.ENTER)
time.sleep(3)

document["GoogleID"]=browser.find_element(By.XPATH, "//span[@id='place-id']").text
print(document["GoogleID"])
#############################################

df_data =pd.DataFrame([document])
df_data.to_csv('data0.csv', mode='a', index=False, header=True)